# Temporal Triangles Neo4j
This notebook walks through running the temporal triangles benchmark on xGT and comparing the performance to Neo4j.
It uses the following data set that would need to be loaded into Neo4j: https://datasets.rocketgraph.com/TT/tt.1M.

In [1]:
from xgt_connector import Neo4jConnector, Neo4jDriver
import xgt
import time

## Connect to xGT and Neo4j

In [2]:
xgt_server = xgt.Connection()
database = "temporal-triangles-1M"
neo4j_server = Neo4jDriver(auth=('neo4j', 'foo'), database=database)
c = Neo4jConnector(xgt_server, neo4j_server)

## Query to run

In [3]:
threshold=15
query = """
    MATCH (v1)-[e1:Edges]->(v2)
              -[e2:Edges]->(v3)
              -[e3:Edges]->(v1)
    WHERE v1 <> v2 AND v1 <> v3 AND v2 <> v3
      AND e1.timestamp <= e2.timestamp
      AND e2.timestamp <= e3.timestamp
      AND e3.timestamp - e1.timestamp < {:d}
    RETURN v1.id as v1id,
           e1.timestamp as e1_timestamp,
           v2.id as v2id,
           e2.timestamp as e2_timestamp,
           v3.id as v3id,
           e3.timestamp as e3_timestamp,
           e3.timestamp - e1.timestamp as delta
  """.format(threshold)

## Transfer data to xGT

In [4]:
t0 = time.time()
c.transfer_to_xgt()
t_duration = time.time() - t0
print(f"Transfer to xgt time: {t_duration:,.2f}")

Transferring: [############################################################] 1090677/1090677 in 0:00:34.7s (31433.6/s, eta: 0:00:00.0s)
Transfer to xgt time: 34.86


## Run query on xGT

In [5]:
t0 = time.time()
job = xgt_server.run_job(query)
q_duration = time.time() - t0
print(job.get_data())
print(f"Query to xgt time: {q_duration:,.2f}")
print(f"Total xgt time including transfer: {t_duration + q_duration:,.2f}")

[[26563, 7598, 51565, 7598, 15727, 7612, 14]]
Query to xgt time: 0.49
Total xgt time including transfer: 35.35


## Run query on Neo4j

In [6]:
t0 = time.time()
with neo4j_server.query(query) as job:
    for row in job.result():
        print(row)
    n_duration = time.time() - t0
    print(f"Neo4j query time: {n_duration:,.2f}")

<Record v1id=26563 e1_timestamp=7598 v2id=51565 e2_timestamp=7598 v3id=15727 e3_timestamp=7612 delta=14>
Neo4j query time: 45.05


In [7]:
print(f"Query speedup: {n_duration/q_duration:,.2f}X")
print(f"Speedup including transfer: {n_duration/(t_duration + q_duration):,.2f}X")


Query speedup: 92.72X
Speedup including transfer: 1.27X
